In [222]:
import requests
import pandas as pd
from datetime import date, timedelta, datetime

In [223]:
BASE = "https://reports.sem-o.com/api/v1"

In [224]:
RESOURCE_NAMES = {
    "DA":   "MarketResult_SEM-DA_PWR-MRC-D",
    "IDA1": "MarketResult_SEM-IDA1_PWR-SEM-GB-D",
    "IDA2": "MarketResult_SEM-IDA2_PWR-SEM-GB-D",
    "IDA3": "MarketResult_SEM-IDA3_PWR-SEM-D",
}

In [225]:
def find_doc_id(market, delivery_day=None):
    resource = RESOURCE_NAMES[market]
    url = (
        f"{BASE}/documents/static-reports"
        f"?DPuG_ID=EA-001&page_size=500&order_by=DESC"
        f"&ExcludeDelayedPublication=0&sort_by=Date"
        f"&ResourceName={resource}"
    )
    r = requests.get(url)
    r.raise_for_status()
    docs = r.json().get("items", [])
    if not docs:
        raise ValueError(f"No documents returned for {market} ({resource})")

    for d in docs:
        auction_date = datetime.fromisoformat(d["Date"]).date()
        if market == "DA":
            #print(f'Auction date = {auction_date}')
            effective_date = auction_date + timedelta(days=1)
        else:
            effective_date = auction_date

        # If user asks for a day, try to match it
        if delivery_day and effective_date == delivery_day:
            return d["_id"]

    # If no delivery_day requested, just return the latest
    if delivery_day is None:
        d = docs[0]
        auction_date = datetime.fromisoformat(d["Date"]).date()
        effective_date = auction_date + (timedelta(days=1) if market == "DA" else timedelta(0))
        print(f"✅ Using latest {market}: delivery {effective_date} id={d['_id']}")
        return d["_id"]

    raise ValueError(f"No {market} doc found for {delivery_day}")


In [226]:
def fetch_prices(doc_id, market):
    url = f"{BASE}/documents/{doc_id}?IST=1"
    r = requests.get(url)
    r.raise_for_status()
    data = r.json()
    records = []
    for row in data.get("rows", []):
        if row[1][0] == "Index prices" and row[1][2] == "EUR":
            for ts, price in zip(row[2], row[3]):
                records.append({
                    "Market": market,
                    "Time": pd.to_datetime(ts),
                    "Price_EUR_MWh": float(price)
                })
    return pd.DataFrame(records)

In [227]:
def fetch_all(delivery_day=None):
    dfs = []
    for market in RESOURCE_NAMES:
        try:
            doc_id = find_doc_id(market, delivery_day)
            dfs.append(fetch_prices(doc_id, market))
        except Exception as e:
            print(f"⚠️ Skipping {market}: {e}")
    return pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()

In [228]:
def summarise(df):
    if df.empty or "Market" not in df.columns:
        raise ValueError("Empty or invalid DataFrame for summarization.")

    # Extract delivery date from timestamp
    df["Date"] = df["Time"].dt.date

    return (
        df.groupby(["Date", "Market"])["Price_EUR_MWh"]
          .agg(
              AvgPrice="mean",
              MinPrice="min",
              MaxPrice="max"
          )
          .reset_index()
    )


In [229]:
if __name__ == "__main__":
    df_all = fetch_all()
    print(df_all.head())
    summary = summarise(df_all)
    print("\nDaily Summary:")
    print(summary)

✅ Using latest DA: delivery 2025-08-27 id=68ac45b69620d9500d44d333
✅ Using latest IDA1: delivery 2025-08-25 id=68ab4c1c9620d9520a1288c6
✅ Using latest IDA2: delivery 2025-08-25 id=68ac18059620d9721d3ab18e
✅ Using latest IDA3: delivery 2025-08-24 id=68ab17609620d96706588052
  Market                Time  Price_EUR_MWh
0     DA 2025-08-25 23:00:00          99.00
1     DA 2025-08-26 00:00:00          91.76
2     DA 2025-08-26 01:00:00          82.75
3     DA 2025-08-26 02:00:00          81.47
4     DA 2025-08-26 03:00:00          83.30

Daily Summary:
         Date Market    AvgPrice  MinPrice  MaxPrice
0  2025-08-24   IDA1  100.815000     93.15   108.480
1  2025-08-24   IDA3   98.356833     70.00   125.492
2  2025-08-25     DA   99.000000     99.00    99.000
3  2025-08-25   IDA1   82.848913      3.54   124.750
4  2025-08-25   IDA2   75.215000     11.25   124.990
5  2025-08-26     DA   99.542174     73.88   146.180


In [230]:
url = ("https://reports.sem-o.com/api/v1/documents/static-reports"
       "?DPuG_ID=EA-001&page_size=5&order_by=DESC&excludeDelayedPublication=0"
       "&sort_by=Date&ResourceName=MarketResult_SEM-DA_PWR-MRC-D")

r = requests.get(url)
print(r.status_code)
print(r.json())

200
{'pagination': {'pageSize': 5, 'currentPage': 1, 'totalItems': 363, 'totalPages': 73, 'sort_by': 'Date', 'order_by': 'DESC'}, 'resourceBaseUri': 'http://reports.sem-o.com/documents', 'items': [{'_id': '68a9a2b69620d96f921299a3', 'DPuG_ID': 'EA-001', 'ReportName': 'ETS Market Results', 'Group': ['Market Data'], 'Dynamic': True, 'ResourceName': 'MarketResult_SEM-DA_PWR-MRC-D+1_20250823100000_20250823102927.csv', 'Date': '2025-08-24T10:00:00', 'DateRetention': '2025-08-23', 'PublishTime': '2025-08-25T00:00:02'}, {'_id': '68a851369620d94188524ea3', 'DPuG_ID': 'EA-001', 'ReportName': 'ETS Market Results', 'Group': ['Market Data'], 'Dynamic': True, 'ResourceName': 'MarketResult_SEM-DA_PWR-MRC-D+1_20250822100000_20250822102924.csv', 'Date': '2025-08-23T10:00:00', 'DateRetention': '2025-08-22', 'PublishTime': '2025-08-24T00:00:03'}, {'_id': '68a6ffb69620d911303a8ae3', 'DPuG_ID': 'EA-001', 'ReportName': 'ETS Market Results', 'Group': ['Market Data'], 'Dynamic': True, 'ResourceName': 'Market

In [231]:
df_all[df_all['Market']=='DA']

,Market,Time,Price_EUR_MWh,Date
0,DA,2025-08-25 23:00:00,99.00,2025-08-25
1,DA,2025-08-26 00:00:00,91.76,2025-08-26
2,DA,2025-08-26 01:00:00,82.75,2025-08-26
3,DA,2025-08-26 02:00:00,81.47,2025-08-26
4,DA,2025-08-26 03:00:00,83.30,2025-08-26
5,DA,2025-08-26 04:00:00,88.00,2025-08-26
6,DA,2025-08-26 05:00:00,101.00,2025-08-26
7,DA,2025-08-26 06:00:00,110.86,2025-08-26
8,DA,2025-08-26 07:00:00,109.09,2025-08-26
9,DA,2025-08-26 08:00:00,106.89,2025-08-26


In [232]:
url = "https://reports.sem-o.com/api/v1/documents/static-reports?DPuG_ID=EA-001&page_size=5&order_by=DESC&ExcludeDelayedPublication=0&sort_by=Date&ResourceName=MarketResult_SEM-DA_PWR-MRC-D"
import requests, pandas as pd
r = requests.get(url).json()
for d in r["items"]:
    print(d["Date"], "=> delivery", pd.to_datetime(d["Date"]).date() + pd.Timedelta(days=1), "publish", d["PublishTime"])

2025-08-26T10:00:00 => delivery 2025-08-27 publish 2025-08-25T10:29:56
2025-08-25T10:00:00 => delivery 2025-08-26 publish 2025-08-24T10:30:02
2025-08-24T10:00:00 => delivery 2025-08-25 publish 2025-08-25T00:00:02
2025-08-23T10:00:00 => delivery 2025-08-24 publish 2025-08-24T00:00:03
2025-08-22T10:00:00 => delivery 2025-08-23 publish 2025-08-23T00:00:03


In [233]:
url = (
        f"{BASE}/documents/static-reports"
        f"?DPuG_ID=EA-001&page_size=500&order_by=DESC"
        f"&ExcludeDelayedPublication=0&sort_by=Date"
        f"&ResourceName=MarketResult_SEM-DA_PWR-MRC-D"
    )
r = requests.get(url)
r.raise_for_status()
docs = r.json().get("items", [])
docs

[{'_id': '68ac45b69620d9500d44d333',
  'DPuG_ID': 'EA-001',
  'ReportName': 'ETS Market Results',
  'Group': ['Market Data'],
  'Dynamic': True,
  'ResourceName': 'MarketResult_SEM-DA_PWR-MRC-D+1_20250825100000_20250825102956.csv',
  'Date': '2025-08-26T10:00:00',
  'DateRetention': '2025-08-25',
  'PublishTime': '2025-08-25T10:29:56'},
 {'_id': '68aaf4369620d920e15095f3',
  'DPuG_ID': 'EA-001',
  'ReportName': 'ETS Market Results',
  'Group': ['Market Data'],
  'Dynamic': True,
  'ResourceName': 'MarketResult_SEM-DA_PWR-MRC-D+1_20250824100000_20250824103002.csv',
  'Date': '2025-08-25T10:00:00',
  'DateRetention': '2025-08-24',
  'PublishTime': '2025-08-24T10:30:02'},
 {'_id': '68a9a2b69620d96f921299a3',
  'DPuG_ID': 'EA-001',
  'ReportName': 'ETS Market Results',
  'Group': ['Market Data'],
  'Dynamic': True,
  'ResourceName': 'MarketResult_SEM-DA_PWR-MRC-D+1_20250823100000_20250823102927.csv',
  'Date': '2025-08-24T10:00:00',
  'DateRetention': '2025-08-23',
  'PublishTime': '2025-0